### PROJECT-3. EDA + Feature Engineering

Цель - Построить модель на основе алгоритмов машинного обучения, которая предсказывает рейтинг отеля.

Поля датасета:

hotel_address — адрес отеля;

review_date — дата, когда рецензент разместил соответствующий отзыв;

average_score — средний балл отеля, рассчитанный на основе последнего комментария за последний год;

hotel_name — название отеля;

reviewer_nationality — страна рецензента;

negative_review — отрицательный отзыв, который рецензент дал отелю;

review_total_negative_word_counts — общее количество слов в отрицательном отзыв;

positive_review — положительный отзыв, который рецензент дал отелю;

review_total_positive_word_counts — общее количество слов в положительном отзыве;

reviewer_score — оценка, которую рецензент поставил отелю на основе своего опыта;

total_number_of_reviews_reviewer_has_given — количество отзывов, которые рецензенты дали в прошлом;

total_number_of_reviews — общее количество действительных отзывов об отеле;

tags — теги, которые рецензент дал отелю;

days_since_review — количество дней между датой проверки и датой очистки;

additional_number_of_scoring — есть также некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.

lat — географическая широта отеля;

lng — географическая долгота отеля.

## Загрузка данных

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re # дополнительная библиотека для очистки строки, созданной из массива
import plotly.express as px
from geopy.geocoders import Nominatim
import category_encoders as ce
import scipy.stats as stats

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sf-booking/hotels_test.csv
/kaggle/input/sf-booking/hotels_train.csv
/kaggle/input/sf-booking/submission.csv


In [2]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [3]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [4]:
# Подгрузим наши данные из соревнования

DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

## Описательный анализ данных

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

В датасете 386803 записей, есть пропуски в признаках lat и lng

In [6]:
df_train.head(2)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097


In [7]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128935 entries, 0 to 128934
Data columns (total 16 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               128935 non-null  object 
 1   additional_number_of_scoring                128935 non-null  int64  
 2   review_date                                 128935 non-null  object 
 3   average_score                               128935 non-null  float64
 4   hotel_name                                  128935 non-null  object 
 5   reviewer_nationality                        128935 non-null  object 
 6   negative_review                             128935 non-null  object 
 7   review_total_negative_word_counts           128935 non-null  int64  
 8   total_number_of_reviews                     128935 non-null  int64  
 9   positive_review                             128935 non-null  object 
 

В датасете 128115 записей, есть пропуски в признаках lat и lng

In [8]:
df_test.head(2)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,tags,days_since_review,lat,lng
0,Via Senigallia 6 20161 Milan Italy,904,7/21/2017,8.1,Hotel Da Vinci,United Kingdom,Would have appreciated a shop in the hotel th...,52,16670,Hotel was great clean friendly staff free bre...,62,1,"[' Leisure trip ', ' Couple ', ' Double Room '...",13 days,45.533137,9.171102
1,Arlandaweg 10 Westpoort 1043 EW Amsterdam Neth...,612,12/12/2016,8.6,Urban Lodge Hotel,Belgium,No tissue paper box was present at the room,10,5018,No Positive,0,7,"[' Leisure trip ', ' Group ', ' Triple Room ',...",234 day,52.385649,4.834443


In [9]:
sample_submission.head(2)

,reviewer_score,id
0,1,488440
1,10,274649


In [10]:
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128935 entries, 0 to 128934
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype
---  ------          --------------   -----
 0   reviewer_score  128935 non-null  int64
 1   id              128935 non-null  int64
dtypes: int64(2)
memory usage: 2.0 MB


In [11]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 18 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               515738 non-null  object 
 1   additional_number_of_scoring                515738 non-null  int64  
 2   review_date                                 515738 non-null  object 
 3   average_score                               515738 non-null  float64
 4   hotel_name                                  515738 non-null  object 
 5   reviewer_nationality                        515738 non-null  object 
 6   negative_review                             515738 non-null  object 
 7   review_total_negative_word_counts           515738 non-null  int64  
 8   total_number_of_reviews                     515738 non-null  int64  
 9   positive_review                             515738 non-null  object 
 

В датасете 515738 строк и 18 столбцов. 
В столбцах lat и lng есть пропуски.

Проверим данные на наличие полных дубликатов:

In [13]:
print('Количество дубликатов: {}'.format(data[data.duplicated()].shape[0]))
#data = data.drop_duplicates() # не удаляем дубликаты, так как потом будут ошибки

Количество дубликатов: 336


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 18 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               515738 non-null  object 
 1   additional_number_of_scoring                515738 non-null  int64  
 2   review_date                                 515738 non-null  object 
 3   average_score                               515738 non-null  float64
 4   hotel_name                                  515738 non-null  object 
 5   reviewer_nationality                        515738 non-null  object 
 6   negative_review                             515738 non-null  object 
 7   review_total_negative_word_counts           515738 non-null  int64  
 8   total_number_of_reviews                     515738 non-null  int64  
 9   positive_review                             515738 non-null  object 
 

In [15]:
data.head(4)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,tags,days_since_review,lat,lng,sample,reviewer_score
0,Via Senigallia 6 20161 Milan Italy,904,7/21/2017,8.1,Hotel Da Vinci,United Kingdom,Would have appreciated a shop in the hotel th...,52,16670,Hotel was great clean friendly staff free bre...,62,1,"[' Leisure trip ', ' Couple ', ' Double Room '...",13 days,45.533137,9.171102,0,0.0
1,Arlandaweg 10 Westpoort 1043 EW Amsterdam Neth...,612,12/12/2016,8.6,Urban Lodge Hotel,Belgium,No tissue paper box was present at the room,10,5018,No Positive,0,7,"[' Leisure trip ', ' Group ', ' Triple Room ',...",234 day,52.385649,4.834443,0,0.0
2,Mallorca 251 Eixample 08008 Barcelona Spain,46,11/26/2015,8.3,Alexandra Barcelona A DoubleTree by Hilton,Sweden,Pillows,3,351,Nice welcoming and service,5,15,"[' Business trip ', ' Solo traveler ', ' Twin ...",616 day,41.393192,2.161520,0,0.0
3,Piazza Della Repubblica 17 Central Station 201...,241,10/17/2015,9.1,Hotel Principe Di Savoia,United States of America,No Negative,0,1543,Everything including the nice upgrade The Hot...,27,9,"[' Leisure trip ', ' Couple ', ' Ambassador Ju...",656 day,45.479888,9.196298,0,0.0


Посмотрим на данные столбцов.

review_date - дата

reviewer_nationality, lat, lng - категориальные признаки.

average_score, review_total_negative_word_counts, review_total_positive_word_counts, total_number_of_reviews_reviewer_has_given, total_number_of_reviews, days_since_review, additional_number_of_scoring - числовые признаки.

hotel_address, hotel_name, negative_review, positive_review, tags - строки.

У нас есть дата отзыва и дата между проверкой и очисткой, скорее всего один нужно будет удалить.
Из hotel_address, negative_review, positive_review, tags можно извлечь дополнительную информацию.
Нужно заполнить пропуски в lat и lng.

In [16]:
#посмотрим статистические характеристики
data.describe()

,additional_number_of_scoring,average_score,review_total_negative_word_counts,total_number_of_reviews,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,lat,lng,sample,reviewer_score
count,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,512470.000000,512470.000000,515738.000000,515738.000000
mean,498.081836,8.397487,18.539450,2743.743944,17.776458,7.166001,49.442439,2.823803,0.749999,6.297672
std,500.538467,0.548048,29.690831,2317.464868,21.804185,11.040228,3.466325,4.579425,0.433014,3.902295
min,1.000000,5.200000,0.000000,43.000000,0.000000,1.000000,41.328376,-0.369758,0.000000,0.000000
25%,169.000000,8.100000,2.000000,1161.000000,5.000000,1.000000,48.214662,-0.143372,0.250000,0.625000
50%,341.000000,8.400000,9.000000,2134.000000,11.000000,3.000000,51.499981,0.010607,1.000000,7.900000
75%,660.000000,8.800000,23.000000,3613.000000,22.000000,8.000000,51.516288,4.834443,1.000000,9.600000
max,2682.000000,9.800000,408.000000,16670.000000,395.000000,355.000000,52.400181,16.429233,1.000000,10.000000


In [17]:
data.describe(include='object')

,hotel_address,review_date,hotel_name,reviewer_nationality,negative_review,positive_review,tags,days_since_review
count,515738,515738,515738,515738,515738,515738,515738,515738
unique,1493,731,1492,227,330011,412601,55242,731
top,163 Marsh Wall Docklands Tower Hamlets London ...,8/2/2017,Britannia International Hotel Canary Wharf,United Kingdom,No Negative,No Positive,"[' Leisure trip ', ' Couple ', ' Double Room '...",1 days
freq,4789,2585,4789,245246,127890,35946,5101,2585


additional_number_of_scoring находится в пределах от 1 до 2682 , среднее значение - 498, медиана - 341. Не нормальное распределение.

average_score находится в пределах от 5.2 до 9.8, среднее значение - 8,397, медиана - 8.4. Не нормальное распределение.

review_total_negative_word_counts распределен в диапазоне от 0 до 408, среднее значение - 18,5, медиана - 9. Не нормальное распределен.

total_number_of_reviews имеет размах от 43 до 16670 отзывов, среднее - 2743,74, медиана - 2134. Не нормально распределен.

review_total_positive_word_counts находится в пределах от 0 до 395 слов, среднее значение - 17,77, медиана - 11. Не нормально распределен.

total_number_of_reviews_reviewer_has_given имеет размах от 1 до 355 отзывов, среднее - 7.16, медиана - 3. Не нормально распределен.

days_since_review находится в пределах от 0 до 730, среднее значение 354,44, медиана - 353. Нет выбросов, не нормально распределен.

lat есть пропуски.

lng есть пропуски.

reviewer_score распределен в диапазоне от 0 до 10, средняя оценка - 6.3, медиана - 7.9. Целевой признак.

hotel_address имеет 1493 уникальных адресов. Частота самого популярного значения - 4789. Из адреса можно извлечь страну и город.

review_date имеет 731 уникальных значений.

hotel_name имеет 1492 уникальных названий. Видимо названия повторяются.

reviewer_nationality - 227 разных стран, больше всего рецензентов из United Kingdom.

### Заполнение пропусков

Заполним пропущенные значения с помощью GeoPy

In [18]:
# создадим новый признак, что был пропуск в признаках lat или lng
data['lat_null'] = data['lat'].apply(lambda x: 1 if np.isnan(x) else 0)
data['lng_null'] = data['lng'].apply(lambda x: 1 if np.isnan(x) else 0)

In [19]:
# сначала получим страну и город из адреса отеля
data['country'] = data['hotel_address'].apply(
    lambda x: x.split(' ')[-1].replace('Kingdom', 'United Kingdom'))
data['city'] = data['hotel_address'].apply(
    lambda x: x.split()[-2] if x.split()[-1] != 'Kingdom'
    else x.split()[-5])
data['city'].value_counts()

London       262301
Barcelona     60149
Paris         59928
Amsterdam     57214
Vienna        38939
Milan         37207
Name: city, dtype: int64

In [20]:
# создаем датафрейм с адресами, где нет координат
df_coord = data[data['lat'].isnull()].groupby('hotel_address').count()[['lat','lng']]
df_coord

,lat,lng
hotel_address,,
20 Rue De La Ga t 14th arr 75014 Paris France,0,0
23 Rue Damr mont 18th arr 75018 Paris France,0,0
4 rue de la P pini re 8th arr 75008 Paris France,0,0
Bail n 4 6 Eixample 08010 Barcelona Spain,0,0
Gr nentorgasse 30 09 Alsergrund 1090 Vienna Austria,0,0
Hasenauerstra e 12 19 D bling 1190 Vienna Austria,0,0
Josefst dter Stra e 10 12 08 Josefstadt 1080 Vienna Austria,0,0
Josefst dter Stra e 22 08 Josefstadt 1080 Vienna Austria,0,0
Landstra er G rtel 5 03 Landstra e 1030 Vienna Austria,0,0


In [ ]:
#geolocator = Nominatim(user_agent = "Test") 
#location = geolocator.geocode("20 Rue De La Ga t 14th arr 75014 Paris France") 
#print(location.address)

In [ ]:
#from geopy.geocoders import Nominatim

#geolocator = Nominatim(user_agent="SF_agent")
#hotel_address = "175 5th Avenue NYC"

#except_ = True

#while except_:
#    try:
#        except_ = False
#        location = geolocator.geocode(hotel_address)
#    except:
#        except_ = True

#print((location.latitude, location.longitude))

In [ ]:
#выполнила код выше в другом месте и перенесла координаты
coordinate = [[48.8870221, 2.3478318], 
           [48.8870221, 2.3478318], 
           [48.8870221, 2.3478318], 
           [41.3936885, 2.1636552],
           [48.22507295, 16.35839764159848],
           [48.2048346, 16.3702081],
           [48.2108519, 16.347359861911986],
           [48.2108519, 16.347359861911986],
           [48.20254735, 16.38461641187194],
           [48.16310865, 16.458012842051907],
           [41.3936885, 2.1636552],
           [48.2149546, 16.302153494876826],
           [41.3936885, 2.1636552],
           [48.2048346, 16.3702081],
           [48.2006384, 16.426895311477978],
           [48.22507295, 16.35839764159848],
           [48.22507295, 16.35839764159848]]

In [ ]:
# Заносим данные в созданный датафрейм.
for i in range(len(df_coord)):
    df_coord.iloc[i] = coordinate[i]
df_coord

In [ ]:
# Переносим данные в датафрейм
data = data.set_index('hotel_address')
data.update(df_coord)
data = data.reset_index()

In [ ]:
data.info()

In [ ]:
data.nunique(dropna=False)

Стран и городов - 6 значений, можно создать новые признаки.

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(), annot=True)

Сильно скоррелированы total_number_of_reviews и additional_number_of_scoring, lat_null и lng_null - один удалим

In [ ]:
data.drop('lng_null', axis = 1, inplace=True)

### Оценка отеля в зависимости от среднего балла отеля, рассчитанного на основе последнего комментария за последний год

In [ ]:
boxplot = sns.boxplot(
    data= data,
    y='average_score',
    x='reviewer_score',
    orient='h',
    width=0.9
)
boxplot.set_title('Распределение оценки отеля по среднему баллу ');
boxplot.set_xlabel('Оценка');
boxplot.set_ylabel('Средний балл');
boxplot.grid()

Чем выше средний балл, тем выше оценка.

In [ ]:
# новый признак на основе average_score
data['average_score_max_norm'] = data['average_score'] / data['city'].map(
    data.groupby(['city'])['average_score'].max())

### Числовые признаки

In [ ]:
# Преобразуем признак days_since_review в числовой
data['days_since_review'] = data['days_since_review'].apply(
    lambda x: int(x.split(' ')[0]))

In [ ]:
# Гистограммы распределения числовых признаков
fig, (ax1) = plt.subplots(1, figsize=(13, 12))
data.hist(ax=ax1)
plt.suptitle('Гистограммы распределения числовых признаков');

На нормальное распределение похож только average_score

In [ ]:
# задаём уровень значимости
alpha = 0.05 

# функция для принятия решения о нормальности
def decision_normality(p):
    print('p-value = {:.3f}'.format(p))
    if p <= alpha:
        print(
            'p-значение меньше, чем заданный уровень значимости {:.2f}. Распределение отлично от нормального'.format(alpha))
    else:
        print(
            'p-значение больше, чем заданный уровень значимости {:.2f}. Распределение является нормальным'.format(alpha))

In [ ]:
num_cols = ['additional_number_of_scoring', 'average_score',
       'review_total_negative_word_counts', 'total_number_of_reviews',
       'review_total_positive_word_counts',
       'total_number_of_reviews_reviewer_has_given', 'lat',
       'lng','days_since_review']

#проверим распределение числовых признаков на нормальность с помощью теста Шапиро — Уилка
for col in num_cols:
    print('Признак ' + col)
    result = stats.shapiro(data[col])
    decision_normality(result[1])

In [ ]:
#from sklearn import preprocessing
# инициализируем нормализатор RobustScaler
#r_scaler = preprocessing.RobustScaler()
# кодируем исходный датасет
#data[num_cols] = r_scaler.fit_transform(data[num_cols])

In [ ]:
# и log и нормализация одинаковый результат дали, оставила нормализацию
#data['additional_number_of_scoring_log'] = np.log2(data['additional_number_of_scoring']+1)
#data['average_score_log'] = np.log2(data['average_score']+1)
#data['review_total_negative_word_counts_log'] = np.log2(data['review_total_negative_word_counts']+1)
#data['total_number_of_reviews_log'] = np.log2(data['total_number_of_reviews']+1)
#data['review_total_positive_word_counts_log'] = np.log2(data['review_total_positive_word_counts']+1)
#data['total_number_of_reviews_reviewer_has_given_log'] = np.log2(data['total_number_of_reviews_reviewer_has_given']+1)       
#data['lat_log'] = np.log2(data['lat']+1)    
#data['lng_log'] = np.log2(data['lng']+1)       
#data['days_since_review_log'] = np.log2(data['days_since_review']+1)   
#data = data.drop(['additional_number_of_scoring', 'average_score', 
#                  'review_total_negative_word_counts', 'total_number_of_reviews', 'review_total_positive_word_counts', 
#                  'total_number_of_reviews_reviewer_has_given', 'lat', 'lng', 'days_since_review'], axis = 1)

### Оценка отеля в зависимости от даты отзыва

In [ ]:
#получить сезон
def get_season(date):
    month = date.month
    if month >= 3 and month <= 5:
        return 2
    elif month >= 6 and month <= 8:
        return 3
    elif month >= 9 and month <= 11:
        return 4
    else:
        return 1

In [ ]:
data['review_date'] = pd.to_datetime(data['review_date'])
data['review_month'] = data['review_date'].dt.month # месяц
data['review_year'] = data['review_date'].dt.year # год
data['review_quarter'] = data['review_date'].dt.quarter #квартал
data['review_day_week'] = data['review_date'].dt.day_of_week #день недели
data['season'] = data['review_date'].apply(get_season) #сезон

In [ ]:
boxplot = sns.boxplot(
    data= data,
    y='review_month',
    x='reviewer_score',
    orient='h',
    width=0.9
)
boxplot.set_title('Распределение оценки отеля по месяцу отзыва ');
boxplot.set_xlabel('Оценка');
boxplot.set_ylabel('Месяц отзыва');
boxplot.grid()

Возможно есть зависимость от месяца отзыва

In [ ]:
boxplot = sns.boxplot(
    data= data,
    y='review_year',
    x='reviewer_score',
    orient='h',
    width=0.9
)
boxplot.set_title('Распределение оценки отеля по году отзыва ');
boxplot.set_xlabel('Оценка');
boxplot.set_ylabel('Год отзыва');
boxplot.grid()

В данных представлены данные за три года, больше всего отзывов в 2016 году

In [ ]:
boxplot = sns.boxplot(
    data= data,
    y='review_quarter',
    x='reviewer_score',
    orient='h',
    width=0.9
)
boxplot.set_title('Распределение оценки отеля по кварталу отзыва ');
boxplot.set_xlabel('Оценка');
boxplot.set_ylabel('Квартал отзыва');
boxplot.grid()

Есть отличия по кварталам, что логично, т.к. уже видели, что есть разница по месяцам

In [ ]:
boxplot = sns.boxplot(
    data= data,
    y='review_day_week',
    x='reviewer_score',
    orient='h',
    width=0.9
)
boxplot.set_title('Распределение оценки отеля по дню недели отзыва ');
boxplot.set_xlabel('Оценка');
boxplot.set_ylabel('Деь недели отзыва');
boxplot.grid()

Тоже есть отличия, но скорее всего это уже лишний признак, т.к. отзыв могут писать просто, когда есть свободное время

In [ ]:
boxplot = sns.boxplot(
    data= data,
    y='season',
    x='reviewer_score',
    orient='h',
    width=0.9
)
boxplot.set_title('Распределение оценки отеля по сезону отзыва ');
boxplot.set_xlabel('Оценка');
boxplot.set_ylabel('Сезон отзыва');
boxplot.grid()

По медиане оценка отеля выше зимой, возможно люди проводят отдых на новогодних праздниках в этих отелях и в это время там все выглядит привлекательнее из-за украшений и новогоднего настроения

### Зависимость оценки отеля от страны и города отеля

In [ ]:
boxplot = sns.boxplot(
    data= data,
    y='country',
    x='reviewer_score',
    orient='h',
    width=0.9
)
boxplot.set_title('Распределение оценки отеля по странам, где расположен отель ');
boxplot.set_xlabel('Оценка');
boxplot.set_ylabel('Страна');
boxplot.grid()

Всего представлено 6 стран в датасете, больше всего отзывов в Италии, Франции и Австрии. По медиане оценки выше в Нидерландах, Испании и Австрии.

In [ ]:
boxplot = sns.boxplot(
    data= data,
    y='city',
    x='reviewer_score',
    orient='h',
    width=0.9
)
boxplot.set_title('Распределение оценки отеля по городам, где расположен отель ');
boxplot.set_xlabel('Оценка');
boxplot.set_ylabel('Город');
boxplot.grid()

В датасете 6 городов. Результаты аналогичны данным по странам. Одной стране соответствует один город. Удалим признак страны из датасета, т.к. название города однозначно определяет, где находится отель, к тому же у нас есть координаты.

In [ ]:
#data.drop('country', axis = 1, inplace=True)

In [ ]:
# преобразуем город в OneHot признак
data = pd.get_dummies(data, columns=['city'])
data.columns

### Оценка отеля в зависимости от названия отеля

In [ ]:
# посмотрим топ-5 отелей
grouped = data.groupby('hotel_name')['reviewer_score'].median().nlargest(7)
fig = plt.figure(figsize=(13, 4))
main_axes = fig.add_axes([0, 0, 1, 1])
main_axes.bar(x=grouped.index, height=grouped);
main_axes.set_ylabel('Оценка')
main_axes.set_title('Топ 5 отелей')

Не информативно, можно часть отелей с высоким рейтингом оставить, а остальное закодировать как "Other"

In [ ]:
data['hotel_name'].head(30)

Есть повторения в названиях, например "Hilton", возможно это сеть отелей

In [ ]:
# взято из Интеренета, топ-10 отелей Европы
hotels_chain_list = ['Marriot', 'International', 'Hilton', 'InterContinental', 'Group',
                     'Inn', 'Accor', 'Best Western', 'Aimbridge', 'GG', 
                     'Hyatt']


# сформируем признак и обозначим отели принадлежащие к сети, как "1", в противном случае "0"
def get_hotel_cain(hotel_name):
    for name_cain in hotels_chain_list:
        if name_cain in hotel_name:
            return 1
    return 0


# выведем результат 
data['chain'] = data['hotel_name'].apply(get_hotel_cain)

In [ ]:
data['chain']

### Оценка отеля в заивисимости от национальности рецензента

In [ ]:
data['reviewer_nationality'].value_counts().tail(60).index

В данных вставлены пробелы в начале строки и в конце, уберем

In [ ]:
data['reviewer_nationality'] = data['reviewer_nationality'].apply(lambda x: x[1:-1])

In [ ]:
data['reviewer_nationality'].value_counts().tail(60).index

In [ ]:
#выведем первые 10 стран с самыми большими оценками
grouped = data.groupby('reviewer_nationality')['reviewer_score'].mean().nlargest(10)
fig = plt.figure(figsize=(13, 4))
main_axes = fig.add_axes([0, 0, 1, 1])
main_axes.bar(x = grouped.index, height = grouped);
main_axes.set_ylabel('Оценка отеля')
main_axes.set_title('Топ 10 стран рецензентов')

Видим место проживания Антарктика, по информации из Интернета, там нет постоянного населения. Значит при заполнении данных произошла ошибка.

In [ ]:
data.groupby('reviewer_nationality')['reviewer_score'].mean().nlargest(10)

In [ ]:
# посмотрим какие отели посетили рецензенты из Антарктики
data[(data['reviewer_nationality']=='Antarctica')]

Заменить непонятно чем можно, возможно все же это какие-то работники станций в Антарктике, которые теперь её считают своим домом. Пока так оставляем

Создадим новый признак - совпадает ли страна рецензента со страной отеля

In [ ]:
data['is_country_reviewer'] = data.apply(
    lambda row: 1 if row['reviewer_nationality'].strip() == row['country'] else 0, axis=1) 

Стран много, поэтому возьмем первые 20, остальные отметим как Other

In [ ]:
popular_nationality = data['reviewer_nationality'].value_counts().nlargest(20).index

In [ ]:
data['reviewer_nationality'] = data['reviewer_nationality'].apply(lambda x: x if x in popular_nationality else 'other')
data['reviewer_nationality'].value_counts()

Больше всего рецензентов из United Kingdom

In [ ]:
# закодируем признак
bin_encoder = ce.BinaryEncoder(cols=['reviewer_nationality']) # указываем столбец для кодирования
type_bin = bin_encoder.fit_transform(data['reviewer_nationality'])
data = pd.concat([data, type_bin], axis=1)

In [ ]:
data.info()

In [ ]:
data.head(4)

### Отрицательные и положительные отзывы

In [ ]:
data['negative_review'].head(10)

Проанализируем отзывы с помощью SentimentIntensityAnalyzer

In [ ]:
# Загрузим корпус данных для анализа тональности:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
nltk.downloader.download('vader_lexicon')

sent_analyzer = SentimentIntensityAnalyzer()

# Используем метод polarity_scores для анализа тональности негативных и позитивных отзывов. 
# Результаты запишем в столбцы neg_sent и pos_sent. Также, для того, чтобы эту информацию можно 
# было использовать в обучении модели, создадим отдельные признаки со значениями neg, neu, pos 
# и compound, которые определяют степень отрицательности, нейтральности, положительности и 
# общую тональность текста соответственно:

data['neg_sent'] = data['negative_review'].apply(lambda x: sent_analyzer.polarity_scores(x))

data['neg_sent']

In [ ]:
data['neg_sent'][0]

In [ ]:
data['neg_sent_neg'] = data['neg_sent'].apply(lambda x: x['neg'])
data['neg_sent_neu'] = data['neg_sent'].apply(lambda x: x['neu'])
data['neg_sent_pos'] = data['neg_sent'].apply(lambda x: x['pos'])
data['neg_sent_cmpnd'] = data['neg_sent'].apply(lambda x: x['compound'])

In [ ]:
data['pos_sent'] = data['positive_review'].apply(lambda x: sent_analyzer.polarity_scores(x))
data['pos_sent_neg'] = data['pos_sent'].apply(lambda x: x['neg'])
data['pos_sent_neu'] = data['pos_sent'].apply(lambda x: x['neu'])
data['pos_sent_pos'] = data['pos_sent'].apply(lambda x: x['pos'])
data['pos_sent_cmpnd'] = data['pos_sent'].apply(lambda x: x['compound'])

In [ ]:
data.info()

In [ ]:
data = data.drop(['neg_sent', 'pos_sent'], axis = 1)

### Тэги

In [ ]:
data['tags'].value_counts().head(10)

In [ ]:
data['tags'].value_counts().tail(10)

Видим на первых 10 как минимум 6 тегов:
- был ли питомец
- тип поездки
- количество отдыхающих
- тип комнаты
- количество ночей
- отправлено ли с мобильного устройства

В записи могут быть не все поля, структура неодинаковая.

In [ ]:
# тип поездки, видим два варианта Leisure trip и Business trip
data['Business_trip'] = data['tags'].apply(lambda x: 1 if 'Business trip' in x else 0)
data['Business_trip'].value_counts()

In [ ]:
ax = sns.countplot(data=data, x=data['Business_trip'], hue='Business_trip')
ax.set(xlabel='Вид поездки', ylabel='Количество')
plt.title('Соотношение бизнес-поездки и для отдыха')
plt.show()

Больше записей для обычного отдыха

In [ ]:
data['with_pet'] = data['tags'].apply(lambda x: 1 if 'With a pet' in x else 0)
data['with_pet'].value_counts()

In [ ]:
ax = sns.countplot(data=data, x=data['with_pet'], hue='with_pet')
ax.set(xlabel='Наличие питомца', ylabel='Количество')
plt.title('Соотношение отдыхающих с питомцем и без')
plt.show()

Мало людей отдыхают с питомцами

In [ ]:
data['mobile'] = data['tags'].apply(lambda x: 1 if 'Submitted from a mobile device' in x else 0)
data['mobile'].value_counts()

In [ ]:
ax = sns.countplot(data=data, x=data['mobile'], hue='mobile')
ax.set(xlabel='Отправлено с мобильного', ylabel='Количество')
plt.title('Зависимость от типа устройства')
plt.show()

С мобильного чаще отправляют

In [ ]:
# количество ночей
reg_stay = "\' Stayed (.*?) night.* \'"  # выделение дня
reg_stay2 = "(\' Stayed .*? night.* \')"  # выделение всего тэга
data['stayed_night'] = data['tags'].apply(lambda x: int(str(re.search(reg_stay, x).group(1))) if re.search(reg_stay, x) else 0)
data['stayed_night'].value_counts()

Чаще всего остаются на 1 ночь

In [ ]:
# посмотрим на график 
table = pd.pivot_table(
                    data,
                    index='stayed_night',
                    values='reviewer_score',
                    aggfunc='mean',
                    )
fig = px.bar(
    data_frame=table,
    title='Средняя оценка отелей в зависимости от продолжительности пребывания',
    width=800,
)
fig.update_layout(showlegend = False)
fig.update_yaxes(range=[5, 9.5]) 
fig.show();

Можно, пожалуй, сказать, что чем дольше человек живет в отеле, тем лучше оценка

In [ ]:
# функция для принятия решения о нормальности
def decision_normality(p):
    print('p-value = {:.3f}'.format(p))
    if p <= alpha:
        print('p-значение меньше, чем заданный уровень значимости {:.2f}. Распределение отлично от нормального'.format(alpha))
    else:
        print('p-значение больше, чем заданный уровень значимости {:.2f}. Распределение является нормальным'.format(alpha))


# функция для принятия решения об отклонении нулевой гипотезы
def decision_hypothesis(p):
    print('p-value = {:.3f}'.format(p))
    if p <= alpha:
        print('p-значение меньше, чем заданный уровень значимости {:.2f}. Отвергаем нулевую гипотезу в пользу альтернативной.'.format(alpha))
    else:
        print('p-значение больше, чем заданный уровень значимости {:.2f}. У нас нет оснований отвергнуть нулевую гипотезу.'.format(alpha))

Проверим зависит зависимы ли оценка отеля и количество отеля

Нулевая гипотеза - признаки зависимы

Альтернативная - признаки не зависимы

In [ ]:
# составляем таблицу сопряжённости
cross_table = pd.crosstab(data['stayed_night'], data['reviewer_score'])
# проводим тест
_, p, _, _ = stats.chi2_contingency(cross_table)
decision_hypothesis(p)

Есть связь между признаком количества ночей и оценкой отеля

In [ ]:
#преобразуем признак tags в список  
data['tags_1'] = data['tags'].apply(
    lambda x: x.replace(
        '[',''
        ).replace(
            ']',''
            ).replace(
                "' ",''
                ).replace(
                    " '",''
                    ).split(', ')
)

data['tags_1'].head()[0]

In [ ]:
# типы рецензентов
type_guest = ['Solo_traveler', 'Couple', 'Group', 'Family_with_young_children', 'Family_with_older_children', 'Travelers_with_friends']

def get_guest(arg):
    for i in arg:
        if i in type_guest:
            return i


In [ ]:
data['type_guest'] = data['tags_1'].apply(get_guest)

In [ ]:
data.head(5)

Не везде указан тип постояльца. 

Закодируем признак.

In [ ]:
encoder = ce.OneHotEncoder(cols=['type_guest']) 
bin = encoder.fit_transform(data['type_guest'])
data = pd.concat([data, bin], axis=1)

In [ ]:
data.head(5)

In [ ]:
data.info()

In [ ]:
# тип комнаты
def room(tags):
    for el in tags:
        if 'Room' in el:
            return el.strip()
    return 'Unknown'

data['room'] = data['tags_1'].apply(room)
data['room'].value_counts()

Получили много значений, оставим 10 часто встречающихся

In [ ]:
popular_room = data['room'].value_counts().head(10)
data['room'] = data['room'].apply(lambda x: x if x in popular_room else 'other')
data['room'].value_counts()

In [ ]:
# преобразуем тип комнаты в OneHot признак
data = pd.get_dummies(data, columns=['room'])

In [ ]:
data.info()

Получили много признаков, нужно часть удалить.

### Матрица корреляции, влияние признаков

In [ ]:
# Построим тепловую матрицу корреляции
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(data.drop(['sample'], axis=1).corr(), annot=True,)

In [ ]:
corr_matrix = data.drop(['sample'], axis=1).corr()
mask = (corr_matrix >= 0.7) | (corr_matrix <= -0.7)
high_corr = corr_matrix[mask]
high_corr = high_corr[high_corr != 1.0].dropna(how='all').dropna(axis=1, how='all')
high_corr = high_corr.stack().reset_index()
high_corr.columns = ['Признак 1', 'Признак 2', 'Значение']
high_corr

In [ ]:
# непрерывные признаки
num_cols = ['additional_number_of_scoring', 'total_number_of_reviews', 'days_since_review', 'review_year',
            'neg_sent_neu', 'neg_sent_neg', 'pos_sent_neu', 'pos_sent_pos']

# категориальные признаки
cat_cols = ['review_month', 'review_quarter']

In [ ]:
X = data.drop(['reviewer_score'], axis = 1)  
y = data['reviewer_score'] 

from sklearn.feature_selection import f_classif # anova

imp_num = pd.Series(f_classif(X[num_cols], y)[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

Удалим additional_number_of_scoring, days_since_review, neg_sent_neg, pos_sent_neu

In [ ]:
y=y.astype('int')

from sklearn.feature_selection import chi2 # хи-квадрат

imp_cat = pd.Series(chi2(X[cat_cols], y)[0], index=cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

Удалим review_quarter

In [ ]:
data = data.drop(['additional_number_of_scoring', 'days_since_review', 
                  'city_Barcelona', 'city_Vienna', 'neg_sent_neg', 
                  'review_quarter', 'pos_sent_neu'], axis = 1)
data.info()

In [ ]:
# удалим признаки, которые не важны для модули (важность см.ниже)
#data = data.drop(['lat_null', 'with_pet', 'type_guest_1', 
#                  'room_Classic Double Room', 'room_Deluxe Double Room', 'room_Double Room',
#                  'room_Double or Twin Room', 'room_Standard Double Room', 'room_Standard Double or Twin Room', 
#                  'room_Standard Twin Room', 'room_Superior Double Room', 'room_Superior Double or Twin Room',
#                  'room_Unknown', 'room_other', 'city_Amsterdam', 'city_London', 'city_Milan', 'city_Paris', 
#                  'chain', 'is_country_reviewer', 'reviewer_nationality_0', 
#                  'reviewer_nationality_2', 'reviewer_nationality_4'], axis = 1)
#data.info()

Ухудшается модель, если удалять признаки

### Строим модель

In [ ]:
# убираем признаки которые еще не успели обработать, 
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
data.drop(object_columns, axis = 1, inplace=True)
data.drop('review_date', axis = 1, inplace=True)

In [ ]:
data.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAPE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(25).plot(kind='barh')

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
list(sample_submission)

In [ ]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

В рамках работы над текущим проектом были выполнены следующие задачи:

- Очистка данных от пропущенных значений 
- Выделение из имеющихся данных новых признаков
- Отбор наиболее значимых признаков
- Построение и обучение модели

Наиболее важными признаками для модели стали - review_total_negative_word_counts, pos_sent_cmpnd, neg_sent_cmpnd, average_score, review_total_posititve_word_counts, average_score_max_norm, total_number_of_review.

Результаты:

- Добились улучшения метрики
- Приняли участие в соревновании